# not using yet

## psf_image_conv_filament
Script that runs the image convolution on the EAGLE projections, specifically the section containing the filament that we cut out.  Use the profiles that are approximately the Dragonfly PSF: double Moffat with Aureole.

Want to convolve the simulation before reducing to the Dragonfly resolution...

__Input:__<br>
paths to .npz files that are both non-SF and SF

__Output:__<br>
maps of the emission with the PSF convolution for SF and nonSF<br>
comparison of SB profiles with the PSF convolution for SF and nonSF

In [1]:
%run 'extract_FOV.ipynb'
### Need to load those previous two before the image convolution one or else will crash (something contradicts in these)
%run 'psf_image_conv.ipynb'

Load the EAGLE simulation

In [2]:
import os

### some functions from load_data
def changeres(distance,resolution,data):
    """
    distance: one of 50Mpc, 100Mpc, 200Mpc, or 500Mpc
    resolution: desired output resolution (e.g. 14 arcsec, 100 arcsec, 500 arcsec, 1000 arcsec)
    data: the raw data (size 32000 x 32000 pixels, 100 Mpc x 100 Mpc)
    """
    pixscale_tomatchres = {'46Mpc': 0.221/1000.*(1.+0.0107),'92Mpc':0.437/1000.*(1.+0.0215)}
    pixscale            = {'50Mpc': 0.237/1000.*(1.+0.0115), '100Mpc': 0.477/1000.*(1.+0.0235),\
                           '200Mpc': 0.928/1000.*(1.+0.047) , '500Mpc': 2.178/1000.*(1.+0.12),\
                           '980Mpc': 3.825/1000.*(1.+0.24)} ### Mpc / arcsec (comoving)
    size = 32000. # pixels 
    scale = 100.  # Mpc 
    
    if distance in pixscale:
        simpixsize = scale/size ### Mpc / pixel is resolution of raw data 
        factor = round(pixscale[distance]*resolution/simpixsize)
        print("Will reduce resolution by a factor of %s."%factor)
        # LATER determine the current resolution of the data. FOR NOW assume current resolution is 100 Mpc/ 32000 pixels ~ 3 kpc/pixel

        'If the factors are not integer multiples of the size (32000), trim the data first and then imreduce it'
        if size%((factor)) != 0.:
            times_factor_fits_in = int(size/factor)
            newsize = times_factor_fits_in * factor
            print("Before reducing resolution, the original data was trimmed to size %s."%newsize)
            datanew = data[0:int(newsize),0:int(newsize)]
        else:
            datanew = data
            newsize = size
    else:
        print('distance needs to be one of: 50Mpc, 100Mpc, 200Mpc, 500Mpc, or 980Mpc.  Others not supported atm.')
        return None
    
    if factor < 2.:
        return datanew, newsize, factor
    else:
        return imreduce(datanew, round(factor), log=True, method = 'average'), newsize, factor


def find_output_resolution(distance,DragonflyTelescope):
    """
    inputs:
    distance: one of 50Mpc, 100Mpc, 200Mpc, or 500Mpc
    data: the raw data (size 32000 x 32000 pixels, 100 Mpc x 100 Mpc)
    """
    pixscale_tomatchres = {'46Mpc': 0.221/1000.*(1.+0.0107),'92Mpc':0.437/1000.*(1.+0.0215)}
    pixscale            = {'50Mpc': 0.237/1000.*(1.+0.0115), '100Mpc': 0.477/1000.*(1.+0.0235),\
                           '200Mpc': 0.928/1000.*(1.+0.047) , '500Mpc': 2.178/1000.*(1.+0.12),\
                           '980Mpc': 3.825/1000.*(1.+0.24)} ### Mpc / arcsec (comoving)
    res_Dragonfly = 2.8 # arcsec / pixel
    ## For now, assuming that the size and scale of the simulation is that of the raw simulation: 100Mpc/32000pixels
    size = 32000. # pixels 
    scale = 100.  # Mpc 
    
    if distance in pixscale:
        simpixsize = scale/size ### Mpc / pixel is physical resolution of raw data
        'calculate the resolution of the raw simulation at the distance specified'
        resolution = simpixsize / pixscale[distance] ### (Mpc / pixel) / (Mpc / arcsec) = arcsec/pixel is angular resolution of raw data
        print("The resolution of the raw simulation at %s is %s."%(distance,resolution))
        
        if resolution < res_Dragonfly and DragonflyTelescope:
            factor = round(res_Dragonfly/resolution)
            print ("%s is less than the Dragonfly resolution (%s) so (if resolution = None) would bin data by %s."\
                   % (resolution, res_Dragonfly, factor))
            resolution = resolution*factor
            print ("Final resolution would be %s" % resolution)
        return resolution
    else:
        print('distance needs to be one of: 50Mpc, 100Mpc, 200Mpc, 500Mpc, or 980Mpc.  Others not supported atm.')
        return None

def loaddata(machine):
    homedir=''
    if machine=='chinook':
        homedir='/Users/lokhorst/Eagle/'
    elif machine=='coho':
        homedir='/Users/deblokhorst/eagle/SlicesFromNastasha/'
    ## Add the path to where the raw data is kept on your computer here
    
    files_SF_28 = [homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen12.5__fromSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen17.5__fromSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen7.5__fromSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen2.5__fromSFR.npz']

    files_noSF_28 = [homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen12.5_noSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen17.5_noSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen7.5_noSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen2.5_noSFR.npz']
    
    file_SF_28 = homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_20slice_total_fromSFR.npz'
    file_noSF_28 = homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_20slice_total_noSFR.npz'

    if os.path.isfile(file_SF_28):
        print 'Total fromSFR file exists, loading now...'
        halpha_fromSF = np.load(file_SF_28)['arr_0']
    else:
        print 'Total fromSFR file doesnt exist, making it now...'
        halpha_fromSF = np.load(files_SF_28[0])['arr_0']
        for filename in files_SF_28[1:]:
            print 'adding: '+filename
            halpha_fromSF = np.log10(10**halpha_fromSF + 10**(np.load(filename)['arr_0']))
        print 'Saving total fromSFR file to: %s'%file_SF_28
        np.savez(file_SF_28,halpha_fromSF)

    if os.path.isfile(file_noSF_28):
        print 'Total noSFR file exists, loading now...'
        halpha_noSF = np.load(file_noSF_28)['arr_0']
    else:
        print 'Total noSFR file doesnt exist, making it now...'
        halpha_noSF = np.load(files_noSF_28[0])['arr_0']
        for filename in files_noSF_28[1:]:
            print 'adding: '+filename
            halpha_noSF = np.log10(10**halpha_noSF + 10**(np.load(filename)['arr_0']))
        print 'Saving total noSFR file to: %s'%file_noSF_28
        np.savez(file_noSF_28,halpha_noSF)

    return halpha_fromSF,halpha_noSF

In [ ]:
halpha_fromSF,halpha_noSF = loaddata('coho')

In [ ]:
halpha_total = np.log10(10**halpha_fromSF + 10**halpha_noSF)

In [ ]:
distance = '50Mpc'; z = 0.0115;
resolution = find_output_resolution(distance,True)
fwhm = 3.0*arcsec

Convolve it with the psf.

Save the psf convolved EAGLE simulation to be loaded into the other script.